In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
# Must set SNORKELDB before importing SnorkelSession
from snorkel import SnorkelSession
from snorkel.parser import TextDocPreprocessor
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence
session = SnorkelSession()

In [2]:
doc_preprocessor = TextDocPreprocessor('small_data_pp/small_match_test.txt')

corpus_parser = CorpusParser()
corpus_parser.apply(doc_preprocessor)

Clearing existing...
Running UDF...
Documents: 1
Sentences: 17


In [2]:
print "Documents:", session.query(Document).count()
print "Sentences:", session.query(Sentence).count()

Documents: 400
Sentences: 95656


In [2]:
from snorkel.candidates import Ngrams, CandidateSpace, CandidateExtractor
from snorkel.models import Document, Sentence, candidate_subclass

GenePhenoPair = candidate_subclass('GPPair',['gene', 'pheno'])

In [3]:
all_sents = set()
docs = session.query(Document).order_by(Document.name).all()
countdev=0
for i, doc in enumerate(docs):
    for s in doc.sentences:
        if i > 4 and i <=10:
            all_sents.add(s)
        if i > 10: break

In [4]:
print len(all_sents)

1152


In [ ]:
from extraction import GM, PM

gene_ngrams = Ngrams(n_max=5)
pheno_ngrams = Ngrams(n_max=10)
cand_extractor = CandidateExtractor(GenePhenoPair, 
                                    [gene_ngrams, pheno_ngrams], [GM, PM],
                                    symmetric_relations=True)

print "Extracting Candidates..."

cand_extractor.apply(all_sents, split=1)
all_cands = session.query(GenePhenoPair).filter(GenePhenoPair.split==1).all()
print "Number of dev candidates:", len(all_cands)

In [ ]:
from snorkel.viewer import SentenceNgramViewer
sv = SentenceNgramViewer(all_cands, session, annotator_name = 'gold')

In [ ]:
sv